<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/NFT_mint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colaboratory用NFT発行プログラム
# ライブラリのインストール
!pip install web3 eth-account

import json
import os
from web3 import Web3
from eth-account import Account
from IPython.display import display, HTML

# Web3接続設定
# Infuraを使用する場合は、プロジェクトIDを入力してください
INFURA_PROJECT_ID = "あなたのInfuraプロジェクトID"
w3 = Web3(Web3.HTTPProvider(f"https://sepolia.infura.io/v3/{INFURA_PROJECT_ID}"))

# ウォレット設定
# 警告: 本番環境では秘密鍵を直接コードに書かないでください
PRIVATE_KEY = "あなたの秘密鍵"  # テスト用の秘密鍵を使用してください
account = Account.from_key(PRIVATE_KEY)
wallet_address = account.address

print(f"ウォレットアドレス: {wallet_address}")
print(f"接続状態: {w3.is_connected()}")

# NFTコントラクトのソースコード (Solidity)
contract_source = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC721/extensions/ERC721URIStorage.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/utils/Counters.sol";

contract MyNFT is ERC721URIStorage, Ownable {
    using Counters for Counters.Counter;
    Counters.Counter private _tokenIds;

    constructor() ERC721("MyNFT", "MNFT") {}

    function mintNFT(address recipient, string memory tokenURI) public onlyOwner returns (uint256) {
        _tokenIds.increment();
        uint256 newItemId = _tokenIds.current();
        _mint(recipient, newItemId);
        _setTokenURI(newItemId, tokenURI);
        return newItemId;
    }
}
"""

# ファイルに保存
with open('MyNFT.sol', 'w') as f:
    f.write(contract_source)

# Solidity コンパイラをインストール
!npm install -g solc

# OpenZeppelinライブラリをインストール
!npm init -y
!npm install @openzeppelin/contracts

# コントラクトをコンパイル
!mkdir -p build
!solcjs --bin --abi --include-path node_modules/ --base-path . -o build MyNFT.sol

# コンパイル後のファイル
print("コンパイル済みファイル:")
!ls build/

# ABI と バイナリを読み込む
with open('./build/MyNFT_sol_MyNFT.abi', 'r') as f:
    contract_abi = json.load(f)

with open('./build/MyNFT_sol_MyNFT.bin', 'r') as f:
    contract_bytecode = '0x' + f.read()

# NFTメタデータの作成
metadata = {
    "name": "My First NFT",
    "description": "This is my first NFT created with Google Colab",
    "image": "https://gateway.pinata.cloud/ipfs/YOURIPFSHASH",  # IPFSにアップロードした画像のURLに置き換え
    "attributes": [
        {
            "trait_type": "Category",
            "value": "Digital Art"
        },
        {
            "trait_type": "Artist",
            "value": "Your Name"
        }
    ]
}

# メタデータをJSONファイルとして保存
with open('metadata.json', 'w') as f:
    json.dump(metadata, f)

# IPFSにメタデータをアップロード (実際のアップロード方法はIPFSサービスによって異なります)
# ここではPinataのAPIを使用する例を示します
!pip install requests

import requests

def upload_to_pinata(file_path):
    # Pinata APIキー (https://app.pinata.cloud/ で取得)
    PINATA_API_KEY = "あなたのPinataAPIキー"
    PINATA_SECRET_API_KEY = "あなたのPinataシークレットキー"

    url = "https://api.pinata.cloud/pinning/pinFileToIPFS"

    headers = {
        'pinata_api_key': PINATA_API_KEY,
        'pinata_secret_api_key': PINATA_SECRET_API_KEY
    }

    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f)}
        response = requests.post(url, files=files, headers=headers)

    return response.json()

# メタデータをIPFSにアップロード
# metadata_ipfs = upload_to_pinata('metadata.json')
# metadata_uri = f"ipfs://{metadata_ipfs['IpfsHash']}"
# print(f"メタデータURI: {metadata_uri}")

# テスト用に固定URIを使用
metadata_uri = "ipfs://QmExample123456789"  # 実際のIPFS URIに置き換えてください

# コントラクトをデプロイ
def deploy_contract():
    contract = w3.eth.contract(abi=contract_abi, bytecode=contract_bytecode)

    # トランザクションのノンス取得
    nonce = w3.eth.get_transaction_count(wallet_address)

    # コントラクトデプロイトランザクションの構築
    tx = contract.constructor().build_transaction({
        'from': wallet_address,
        'nonce': nonce,
        'gas': 4000000,
        'gasPrice': w3.to_wei('50', 'gwei')
    })

    # トランザクション署名
    signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)

    # トランザクション送信
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    print(f"デプロイトランザクションハッシュ: {tx_hash.hex()}")

    # トランザクション完了を待機
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"コントラクトアドレス: {tx_receipt.contractAddress}")

    return tx_receipt.contractAddress

# NFTのミント関数
def mint_nft(contract_address, recipient_address, token_uri):
    contract = w3.eth.contract(address=contract_address, abi=contract_abi)

    nonce = w3.eth.get_transaction_count(wallet_address)

    # ミントトランザクションの構築
    tx = contract.functions.mintNFT(recipient_address, token_uri).build_transaction({
        'from': wallet_address,
        'nonce': nonce,
        'gas': 2000000,
        'gasPrice': w3.to_wei('50', 'gwei')
    })

    # トランザクション署名
    signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)

    # トランザクション送信
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    print(f"ミントトランザクションハッシュ: {tx_hash.hex()}")

    # トランザクション完了を待機
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    return tx_receipt

# メイン実行部分
# 実行時には INFURA_PROJECT_ID と PRIVATE_KEY を設定してください
if __name__ == "__main__":
    if w3.is_connected():
        # コントラクトのデプロイ
        contract_address = deploy_contract()

        # NFTのミント
        recipient = wallet_address  # 自分自身のアドレスにミント
        mint_receipt = mint_nft(contract_address, recipient, metadata_uri)

        # 結果表示
        print("\nNFT発行完了!")
        print(f"コントラクトアドレス: {contract_address}")
        print(f"NFT所有者: {recipient}")
        print(f"トランザクションハッシュ: {mint_receipt['transactionHash'].hex()}")
        print(f"Etherscanで確認: https://sepolia.etherscan.io/tx/{mint_receipt['transactionHash'].hex()}")

        # NFT情報を表示
        display(HTML(f"""
        <div style="border: 1px solid #ddd; padding: 20px; border-radius: 10px;">
            <h2>NFT発行完了</h2>
            <p><strong>コントラクトアドレス:</strong> {contract_address}</p>
            <p><strong>NFT所有者:</strong> {recipient}</p>
            <p><strong>メタデータURI:</strong> {metadata_uri}</p>
            <p><a href="https://sepolia.etherscan.io/tx/{mint_receipt['transactionHash'].hex()}" target="_blank">Etherscanで確認</a></p>
        </div>
        """))
    else:
        print("Web3接続に失敗しました。Infura Project IDを確認してください。")